### Import libraries

In [50]:
from IPython.core.display import display
from src.utils.preprocessing import preprocessing_pipeline
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [52]:
DATA_PATH = '../../../data'
X_coarse, y_coarse = import_data(DATA_PATH, segmentation_type='coarse',
                                 drop_user_features=True,
                                 drop_expert=True)

In [53]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='coarse',
                       drop_user_features=True,
                       drop_expert=False)

In [54]:
display(X_coarse.head())
display(y_coarse.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               8.0          8.0   
                                     1               2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0               4.0          4.0   
                                     1               3.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0               6.0          6.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          6.0   
                                     1                2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                4.0          4.0   
                                     1                5.0          6.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          9.0   
                                     1                2.0          2.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                6.0          6.0   
                                     1                4.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          7.0   
                                     1                3.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0   
                                     1                6.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          6.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          8.0  ...   
                                     1                3.0          3.0  ...   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0  ...   
                                     1                5.0          4.0  ...   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                8.0          6.0  ...   

                                              MFCC_std11  MFCC_std12  \
subject                              file_id                           
008ba489-31ad-44d8-856b-fcf72369dc46 0         14.163379   10.952018   
                                     1         11.684959    9.106066   
008c1c9e-aeef-40c5-846c-24f1b964f884 0         13.539921    8.786539   
                                     1         12.846291    7.035100   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0         14.501091    9.551842   

                                              Crest_Factor  Cough_Length  \
subject                              file_id                               
008ba489-31ad-44d8-856b-fcf72369dc46 0            8.365480      1.036563   
                                     1            9.348207      0.378625   
008c1c9e-aeef-40c5-846c-24f1b964f884 0            5.222938      0.765625   
                                     1            4.402501      0.794125   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0            4.234178      0.860250   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.314239     0.027049   
                                     1           0.093550     0.107780   
008c1c9e-aeef-40c5-846c-24f1b964f884 0           0.160032     0.034572   
               

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
008c1c9e-aeef-40c5-846c-24f1b964f884 0          1.0
                                     1          1.0
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0          1.0

### Preprocessing

In [55]:
X_coarse = preprocessing_pipeline(X_tr=X_coarse, stop=None, dummy=False)

In [56]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = preprocessing_pipeline(X_e, stop=None, dummy=False)
X_e['Expert'] = expert.values

### Split expert models

In [57]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

In [58]:
DECISION_METRIC = 'roc_auc_score'
CV_K = 10

#### 1. Logistic regression

In [59]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_iter oversampling                                               
10000    True          0.448957       0.603175           0.000163   
         False         0.170428       0.531220           0.000059   
100000   True          0.448957       0.603175           0.000163   
         False         0.170428       0.531220           0.000059   

                       accuracy_score  explained_variance_score  
max_iter oversampling                                            
10000    True                0.606589                 -0.843521  
         False               0.728488                 -0.154657  
100000   True                0.606589                 -0.843521  
         False               0.728488                 -0.154657

In [60]:
# Best model parameters

best_log = log_results.iloc[[
    log_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.448957,0.603175,0.000163,0.606589,-0.843521


In [61]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)

In [62]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.369138,0.656366,0.001356,0.721173,-1.055053


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.301329,0.554153,0.001046,0.601596,-1.404462


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,False,0.608526,0.630547,0.000693,0.631143,-0.473886


#### 2. Linear Discriminant Analysis

In [63]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.450221,0.603876,0.000158,0.605329,-0.842842
False,0.167864,0.530537,0.000058,0.728295,-0.153934


In [64]:
# Best model parameters

best_lda = lda_results.iloc[[
    lda_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.450221,0.603876,0.000158,0.605329,-0.842842


In [65]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LDA_PARAMS)

In [66]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.380967,0.672532,0.001921,0.707166,-1.083197


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.298166,0.549586,0.001523,0.589096,-1.445085


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.622405,0.628547,0.000326,0.628,-0.486653


#### 3. K-nearest Neighbors

In [67]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.450140       0.602536           0.000179   
            False         0.412175       0.597416           0.000141   
2           True          0.432639       0.600448           0.000343   
            False         0.221385       0.541712           0.000093   
3           True          0.450193       0.589365           0.000127   
            False         0.349757       0.573669           0.000126   
4           True          0.442436       0.592503           0.000206   
            False         0.200617       0.536073           0.000159   
5           True          0.444783       0.576675           0.000128   
            False         0.286918       0.551759           0.000271   
6           True          0.436334       0.577377           0.000142   
            False         0.180677       0.531062           0.000180   
7           True          0.442242       0.570134           0.000166   
            False         0.241676       0.538523           0.000222   
8           True          0.437797       0.573599           0.000221   
            False         0.159668       0.525830           0.000110   
9           True          0.444964       0.570959           0.000151   
            False         0.215925       0.533653           0.000113   
10          True          0.442337       0.574588           0.000156   
            False         0.145513       0.522882           0.000041   
11          True          0.440864       0.563641           0.000107   
            False         0.195246       0.531637           0.000089   
12          True          0.440073       0.568876           0.000100   
            False         0.132354       0.521272           0.000042   
13          True          0.442852       0.564061           0.000164   
            False         0.172186       0.527311           0.000115   
14          True          0.440180       0.566536           0.000276   
            False         0.116103       0.517888           0.000057   
15          True          0.441232       0.560798           0.000179   
            False         0.149104       0.523090           0.000068   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.596899                 -0.854378  
            False               0.680329                 -0.624469  
2           True                0.643895                 -0.768486  
            False               0.724322                 -0.229833  
3           True                0.532267                 -0.880041  
            False               0.699031                 -0.501081  
4           True                0.578876                 -0.901119  
            False               0.723256                 -0.219708  
5           True                0.496512                 -0.868263  
            False               0.703779                 -0.429832  
6           True                0.534496                 -0.945244  
            False               0.723256                 -0.202893  
7           True                0.477713                 -0.848405  
            False               0.706492                 -0.376139  
8           True                0.512112                 -0.927248  
            False               0.723159                 -0.186403  
9           True                0.468798                 -0.806929  
            False               0.711434                 -0.320237  
10          True                0.497771                 -0.884718  
            False               0.723740                 -0.170235  
11          True                0.454651                 -0.799890  
            False               0.718217                 -0.253242  
12          True                0.481589                 -0.87030

In [68]:
# Best model parameters

best_knn = knn_results.iloc[[
    knn_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.45014,0.602536,0.000179,0.596899,-0.854378


In [69]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=KNN_PARAMS)

In [70]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.363404,0.659157,0.001144,0.680456,-1.20615


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
2,True,0.310349,0.56934,0.000527,0.658511,-1.197215


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
3,False,0.617849,0.643908,0.000272,0.644571,-0.416249


#### 4. Support Vector Classifier

In [71]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  roc_auc_score_var  \
kernel gamma oversampling                                               
linear 0.10  True          0.453609       0.604691           0.000203   
             False         0.000000       0.500000           0.000000   
rbf    0.10  True          0.153029       0.531673           0.000031   
             False         0.017757       0.503277           0.000010   
linear 0.01  True          0.453609       0.604691           0.000203   
             False         0.000000       0.500000           0.000000   
rbf    0.01  True          0.441761       0.603406           0.000194   
             False         0.002215       0.500356           0.000001   

                           accuracy_score  explained_variance_score  
kernel gamma oversampling                                            
linear 0.10  True                0.595930                 -0.848945  
             False               0.731008                  0.000000  
rbf    0.10  True                0.736143                 -0.080774  
             False               0.731589                 -0.008534  
linear 0.01  True                0.595930                 -0.848945  
             False               0.731008                  0.000000  
rbf    0.01  True                0.631395                 -0.797979  
             False               0.731008                 -0.001545

In [72]:
# Best model parameters

best_svc = svc_results.iloc[[
    svc_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
linear,0.1,True,0.453609,0.604691,0.000203,0.59593,-0.848945


In [73]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=SVC_PARAMS)

In [74]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()[DECISION_METRIC].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
linear,0.1,True,0.365848,0.654488,0.001308,0.714332,-1.085648


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
linear,0.1,True,0.310932,0.561231,0.000906,0.582713,-1.411778


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.1,True,0.568022,0.651697,0.000602,0.655429,-0.254891


#### 5. Naive Bayes Classifier

In [75]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.099235,0.513705,0.000656,0.684690,-0.223863
False,0.088540,0.512813,0.000655,0.713372,-0.183421


In [76]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.099235,0.513705,0.000656,0.68469,-0.223863


In [77]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)

In [78]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.307222,0.598367,0.001093,0.689251,-1.281494


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.294965,0.529018,0.0008,0.471809,-1.550282


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.185035,0.521429,0.000791,0.541143,-0.233189


#### 6. Decision Tree

In [79]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_depth oversampling                                               
3         True          0.404446       0.549765           0.000187   
          False         0.050187       0.504344           0.000123   
5         True          0.414617       0.556365           0.000350   
          False         0.183122       0.522103           0.000378   
7         True          0.401059       0.554158           0.000214   
          False         0.245407       0.530270           0.000374   

                        accuracy_score  explained_variance_score  
max_depth oversampling                                            
3         True                0.520640                 -1.020600  
          False               0.723740                 -0.095173  
5         True                0.520252                 -1.011049  
          False               0.703295                 -0.350121  
7         True                0.552422                 -1.046656  
          False               0.685174                 -0.521899

In [80]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
5,True,0.414617,0.556365,0.00035,0.520252,-1.011049


In [81]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=DECISION_TREE_PARAMS)

In [82]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
5,True,0.346085,0.636091,0.001636,0.704235,-1.164205


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.266734,0.51595,0.001397,0.553191,-1.591528


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
3,True,0.55514,0.60377,0.000614,0.609429,-0.524916


#### 7. Random Forest

In [83]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.401601       0.555579   
                       False         0.030998       0.504816   
          5            True          0.418825       0.575176   
                       False         0.016740       0.502386   
          7            True          0.408777       0.570216   
                       False         0.007183       0.500631   
5         3            True          0.396608       0.558124   
                       False         0.102598       0.512600   
          5            True          0.409355       0.569925   
                       False         0.072439       0.509329   
          7            True          0.409849       0.574749   
                       False         0.069239       0.511866   
7         3            True          0.393424       0.558872   
                       False         0.172144       0.519679   
          5            True          0.396714       0.566819   
                       False         0.152420       0.522459   
          7            True          0.393912       0.566748   
                       False         0.121545       0.519244   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True                   0.000544        0.554748   
                       False                  0.000016        0.730620   
          5            True                   0.000096        0.581880   
                       False                  0.000021        0.730523   
          7            True                   0.000457        0.587984   
                       False                  0.000003        0.730233   
5         3            True                   0.000315        0.576357   
                       False                  0.000076        0.721899   
          5            True                   0.000217        0.586919   
                       False                  0.000053        0.725775   
          7            True                   0.000150        0.602616   
                       False                  0.000039        0.731008   
7         3            True                   0.000185        0.586628   
                       False                  0.000070        0.707558   
          5            True                   0.000273        0.605717   
                       False                  0.000049        0.720155   
          7            True                   0.000303        0.611628   
                       False                  0.000110        0.725678   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -1.024678  
                       False                        -0.026458  
          5            True                         -0.957104  
                       False                        -0.016450  
          7            True                         -0.956403  
                       False                        -0.011231  
5         3            True                         -1.012512  
                       False                        -0.149415  
          5            True                         -0.964463  
                       False                        -0.095444  
          7            True                         -0.925422  
                       False                        -0.052971  
7         3            True                         -0.995651  
                       False                        -0.314507  
          5            True                         -0.937248  
                       False                        -0.203208  
          7            True                         -0.920468  
                       False                        -0.1355

In [84]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,5,True,0.418825,0.575176,0.000096,0.58188,-0.957104


In [85]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)

In [86]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.356501,0.640505,0.000773,0.734528,-1.015553


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,5,True,0.283418,0.533518,0.001045,0.557979,-1.525398


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,7,True,0.599263,0.62764,0.000418,0.629429,-0.473155


#### 8. Gradient Boosting

In [87]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_coarse,
                                      labels=y_coarse.Label, cv_k=CV_K,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.412617       0.565204   
                       False         0.000000       0.500000   
          5            True          0.418072       0.570939   
                       False         0.000000       0.499934   
          7            True          0.421102       0.577398   
                       False         0.000000       0.499802   
5         3            True          0.410343       0.568616   
                       False         0.000000       0.499936   
          5            True          0.414335       0.576274   
                       False         0.007154       0.501021   
          7            True          0.410441       0.575329   
                       False         0.026117       0.504520   
7         3            True          0.402259       0.567660   
                       False         0.002174       0.500416   
          5            True          0.400398       0.570147   
                       False         0.031300       0.504283   
          7            True          0.409033       0.579215   
                       False         0.054688       0.507309   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True               2.414382e-04        0.560174   
                       False              0.000000e+00        0.731008   
          5            True               1.649323e-04        0.568120   
                       False              3.968191e-08        0.730911   
          7            True               1.206199e-04        0.584205   
                       False              1.798146e-07        0.730717   
5         3            True               2.744055e-04        0.580233   
                       False              3.736449e-08        0.730911   
          5            True               2.732199e-04        0.598256   
                       False              3.155267e-06        0.730814   
          7            True               2.420848e-04        0.604167   
                       False              1.625479e-05        0.731395   
7         3            True               1.528811e-04        0.596221   
                       False              1.282826e-06        0.731105   
          5            True               1.165940e-04        0.608818   
                       False              2.110228e-05        0.729554   
          7            True               3.452786e-04        0.621221   
                       False              7.495632e-05        0.727907   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -0.999682  
                       False                         0.000000  
          5            True                         -0.981431  
                       False                        -0.000756  
          7            True                         -0.944169  
                       False                        -0.002278  
5         3            True                         -0.979452  
                       False                        -0.000777  
          5            True                         -0.931343  
                       False                        -0.006866  
          7            True                         -0.921128  
                       False                        -0.016305  
7         3            True                         -0.954852  
                       False                        -0.000827  
          5            True                         -0.921780  
                       False                        -0.034401  
          7            True                         -0.870232  
                       False                        -0.0649

In [88]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,True,0.409033,0.579215,0.000345,0.621221,-0.870232


In [89]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=CV_K,
    params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=CV_K,
    params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=CV_K,
    params=GRADIENT_BOOSTING_PARAMS)

In [90]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()[DECISION_METRIC].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.373081,0.652145,0.001677,0.749511,-0.911941


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,7,True,0.290834,0.541158,0.0008,0.577128,-1.487318


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.598296,0.629824,0.000776,0.631143,-0.466773


### Results

In [91]:
# display('logistic', best_log)
# display('lda', best_lda)
# display('knn', best_knn)
# display('svc', best_svc)
# display('naive_bayes', best_nb)
# display('decision_tree', best_dt)
# display('random_forest', best_rf)
# display('gradient_boosting', best_gb)

In [92]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [93]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
# display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
# display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
# display('knn', pd.concat([best_knn_1,best_knn_2,best_knn_3]))
display('svc', pd.concat([best_svc_1, best_svc_2, best_svc_3]))
# display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
# display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
# display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
# display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'svc'

f1_score  roc_auc_score  roc_auc_score_var  \
kernel gamma oversampling                                               
linear 0.1   True          0.365848       0.654488           0.001308   
             True          0.310932       0.561231           0.000906   
rbf    0.1   True          0.568022       0.651697           0.000602   

                           accuracy_score  explained_variance_score  
kernel gamma oversampling                                            
linear 0.1   True                0.714332                 -1.085648  
             True                0.582713                 -1.411778  
rbf    0.1   True                0.655429                 -0.254891

In [94]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.603175,0.610414
1,lda,0.603876,0.612877
2,knn,0.602536,0.621299
3,svc,0.604691,0.619601
4,naive_bayes,0.513705,0.547049
5,decision_tree,0.556365,0.581410
6,random_forest,0.575176,0.597206
7,gradient_boosting,0.579215,0.604185


### Conclusions


In [95]:
# Save dataframe
results.to_pickle("results_coarse_no_metadata.pkl")

In [96]:
results.mean(axis=0)

auc_best               0.579842
auc_expert_weighted    0.599255
dtype: float64

In [97]:
results.loc[results['auc_best'].argmax()]

models                      svc
auc_best               0.604691
auc_expert_weighted    0.619601
Name: 3, dtype: object

In [98]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.602536
auc_expert_weighted    0.621299
Name: 2, dtype: object